In [1]:
#ライブラリを読み込む
from time import sleep
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#複数ページから情報を取得する
url = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13109&sc=13110&sc=13112&sc=13114&sc=13115&sc=13116&sc=13117&cb=0.0&ct=20.0&et=5&md=07&md=10&cn=5&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page={}'

#変数d_listに空のリストを作成する
d_list = []

In [3]:
#1~2336をループする
for i in range(1,4):
    #変数target_urlに、アクセス先のURLを格納する
    target_url = url.format(i)
   
    #target_URLへのアクセス結果を変数rに格納
    r =requests.get(target_url)
    
    #サーバーに負荷をかけないために、１秒おきにfor文を実行する
    sleep(1)

    #取得結果を解析してsoupに格納
    soup = BeautifulSoup(r.text)

    #すべての物件情報を取得する
    contents = soup.find_all("div",class_="cassetteitem")

    #各物件情報をforループで取得する
    for content in contents:
       #物件情報・部屋情報を取得する
       detail = content.find("div",class_="cassetteitem-detail")
       table = content.find("table",class_="cassetteitem_other")
       #物件情報を取得する
       title = detail.find("div",class_="cassetteitem_content-title").text 
       address = detail.find("li",class_="cassetteitem_detail-col1").text
       accsess = detail.find("li",class_="cassetteitem_detail-col2").text
       age,floor = detail.find("li",class_="cassetteitem_detail-col3").text.split()
       #物件情報から各部屋の情報を取得する
       tr_tags = table.find_all("tr",class_="js-cassette_link")
       for tr_tag in tr_tags:
          #部屋情報の行から欲しい情報を取得する
          floor,price,first_fee,capacity = tr_tag.find_all("td")[2:6]
          #更に細かい情報を取得する
          fee,management_fee = price.find_all("li")
          deposit,gratuity = first_fee.find_all("li")
          madori,menseki = capacity.find_all("li")
          #取得した情報を辞書に格納する
          d = {
             "title":title,
             "address":address,
             "accsess":accsess,
             "age":age,
             "floor":floor.text,
             "fee":fee.text,
             "management_fee":management_fee.text,
             "deposit":deposit.text,
             "gratuity":gratuity.text,
             "madori":madori.text,
             "menseki":menseki.text
             }
          #取得した辞書をd_listに格納する
          d_list.append(d)


In [4]:
#変数d=listを使って、データフレームを作成する
df = pd.DataFrame(d_list)
df

,title,address,accsess,age,floor,fee,management_fee,deposit,gratuity,madori,menseki
0,モデリアブリュット湯島,東京都文京区湯島３,\n東京メトロ千代田線/湯島駅 歩5分\n東京メトロ銀座線/末広町駅 歩5分\nＪＲ総武線/...,新築,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,18万円,15000円,18万円,-,2LDK,44.97m2
1,東京メトロ千代田線 湯島駅 地下1地上9階建 新築,東京都文京区湯島３,\n東京メトロ千代田線/湯島駅 歩5分\n東京メトロ銀座線/末広町駅 歩5分\nＪＲ総武線/...,新築,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,18万円,15000円,18万円,-,2LDK,44.97m2
2,ウェルスクエア南大塚,東京都豊島区南大塚２,\nＪＲ山手線/大塚駅 歩7分\n東京メトロ丸ノ内線/新大塚駅 歩5分\nＪＲ山手線/巣鴨駅...,新築,\r\n\t\t\t\t\t\t\t\t\t\t\t3階,17.2万円,12000円,17.2万円,17.2万円,2LDK,40.59m2
3,ワールドレジデンス,東京都豊島区長崎５,\n西武池袋線/東長崎駅 歩5分\n東京メトロ副都心線/千川駅 歩14分\n都営大江戸線/落...,新築,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,13.6万円,3000円,13.6万円,13.6万円,2LDK,38.97m2
4,ワールドレジデンス,東京都豊島区長崎５,\n西武池袋線/東長崎駅 歩5分\n東京メトロ副都心線/千川駅 歩14分\n都営大江戸線/落...,新築,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,13.6万円,3000円,13.6万円,13.6万円,2LDK,38.97m2
...,...,...,...,...,...,...,...,...,...,...,...
167,東急大井町線 尾山台駅 2階建 築3年,東京都世田谷区等々力２,\n東急大井町線/尾山台駅 歩5分\n東急大井町線/等々力駅 歩7分\n東急大井町線/九品仏...,築3年,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,18.5万円,3000円,-,18.5万円,2LDK,56.26m2
168,京王線 八幡山駅 3階建 新築,東京都杉並区上高井戸１,\n京王線/八幡山駅 歩2分\n京王線/芦花公園駅 歩10分\n京王線/上北沢駅 歩13分\n,新築,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,18.7万円,5000円,18.7万円,18.7万円,2LDK,53.92m2
169,ロジェ代田,東京都世田谷区代田４,\n京王井の頭線/東松原駅 歩5分\n京王井の頭線/新代田駅 歩8分\n小田急線/世田谷代田...,築3年,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,15.5万円,5500円,-,15.5万円,2LDK,40.86m2
170,京王井の頭線 東松原駅 2階建 築3年,東京都世田谷区代田４,\n京王井の頭線/東松原駅 歩5分\n京王井の頭線/新代田駅 歩8分\n京王線/明大前駅 歩...,築3年,\r\n\t\t\t\t\t\t\t\t\t\t\t1階,15.5万円,5500円,-,15.5万円,2LDK,40.86m2


In [5]:
#データクレンジングを実施する
# age列の処理: 築年数のみを抽出し、新築の場合は0に設定
import re
df['age'] = df['age'].apply(lambda x: 0 if '新築' in x else (int(re.search(r'築(\d+)年', x).group(1)) if re.search(r'築(\d+)年', x) else 0))

# floor列の処理: 階数の整数部分を抽出
def floor_number(i):
    if '-' in i:  # メゾネットタイプの場合
        return i.split('階')[0]
    elif 'B' in i:  # 地下の場合
        return int(re.search(r'B(\d+)', i).group(1)) * -1
    else:
        return int(re.search(r'(\d+)階', i).group(1))

df['floor'] = df['floor'].apply(floor_number)

# 金額関連の列(fee, deposit, gratuity)の処理: 万円を整数に変換
def yen_int(yen):
    if yen in ['-', '']:
        return 0
    search = re.search(r'([\d.]+)万?円', yen) 
    if search:
        return int(float(search.group(1)) * 10000)
    return 0  # 万円の形式に一致しない場合は0を返す

# management_fee 列の処理: 万円を整数に変換しない
def yen_int_no_conversion(yen):
    if yen in ['-', '']:
        return 0
    search = re.search(r'([\d.]+)万?円', yen) 
    if search:
        return int(float(search.group(1)))  # 万円を整数に変換せずそのまま返す
    return 0  # 万円の形式に一致しない場合は0を返す

df['fee'] = df['fee'].apply(yen_int)
df['management_fee'] = df['management_fee'].apply(yen_int_no_conversion)  # management_fee 列だけ別の処理を適用
df['deposit'] = df['deposit'].apply(yen_int)
df['gratuity'] = df['gratuity'].apply(yen_int)

# area列の処理: 数値のみを抽出
df['menseki_m2'] = df['menseki'].apply(lambda x: float(re.search(r'([\d.]+)m2', x).group(1)))
df = df.drop("menseki",axis=1)


In [6]:
df

,title,address,accsess,age,floor,fee,management_fee,deposit,gratuity,madori,menseki_m2
0,モデリアブリュット湯島,東京都文京区湯島３,\n東京メトロ千代田線/湯島駅 歩5分\n東京メトロ銀座線/末広町駅 歩5分\nＪＲ総武線/...,0,1,180000,15000,180000,0,2LDK,44.97
1,東京メトロ千代田線 湯島駅 地下1地上9階建 新築,東京都文京区湯島３,\n東京メトロ千代田線/湯島駅 歩5分\n東京メトロ銀座線/末広町駅 歩5分\nＪＲ総武線/...,0,1,180000,15000,180000,0,2LDK,44.97
2,ウェルスクエア南大塚,東京都豊島区南大塚２,\nＪＲ山手線/大塚駅 歩7分\n東京メトロ丸ノ内線/新大塚駅 歩5分\nＪＲ山手線/巣鴨駅...,0,3,172000,12000,172000,172000,2LDK,40.59
3,ワールドレジデンス,東京都豊島区長崎５,\n西武池袋線/東長崎駅 歩5分\n東京メトロ副都心線/千川駅 歩14分\n都営大江戸線/落...,0,1,136000,3000,136000,136000,2LDK,38.97
4,ワールドレジデンス,東京都豊島区長崎５,\n西武池袋線/東長崎駅 歩5分\n東京メトロ副都心線/千川駅 歩14分\n都営大江戸線/落...,0,1,136000,3000,136000,136000,2LDK,38.97
...,...,...,...,...,...,...,...,...,...,...,...
167,東急大井町線 尾山台駅 2階建 築3年,東京都世田谷区等々力２,\n東急大井町線/尾山台駅 歩5分\n東急大井町線/等々力駅 歩7分\n東急大井町線/九品仏...,3,1,185000,3000,0,185000,2LDK,56.26
168,京王線 八幡山駅 3階建 新築,東京都杉並区上高井戸１,\n京王線/八幡山駅 歩2分\n京王線/芦花公園駅 歩10分\n京王線/上北沢駅 歩13分\n,0,1,187000,5000,187000,187000,2LDK,53.92
169,ロジェ代田,東京都世田谷区代田４,\n京王井の頭線/東松原駅 歩5分\n京王井の頭線/新代田駅 歩8分\n小田急線/世田谷代田...,3,1,155000,5500,0,155000,2LDK,40.86
170,京王井の頭線 東松原駅 2階建 築3年,東京都世田谷区代田４,\n京王井の頭線/東松原駅 歩5分\n京王井の頭線/新代田駅 歩8分\n京王線/明大前駅 歩...,3,1,155000,5500,0,155000,2LDK,40.86


In [7]:
len(df)

172

In [8]:
# データフレームから重複を削除
df_ = df.drop_duplicates(['title','address', 'accsess', 'age', 'floor', 'fee', 'management_fee', 'deposit', 'gratuity', 'madori', 'menseki_m2'])


In [9]:
len(df_)
df_

,title,address,accsess,age,floor,fee,management_fee,deposit,gratuity,madori,menseki_m2
0,モデリアブリュット湯島,東京都文京区湯島３,\n東京メトロ千代田線/湯島駅 歩5分\n東京メトロ銀座線/末広町駅 歩5分\nＪＲ総武線/...,0,1,180000,15000,180000,0,2LDK,44.97
1,東京メトロ千代田線 湯島駅 地下1地上9階建 新築,東京都文京区湯島３,\n東京メトロ千代田線/湯島駅 歩5分\n東京メトロ銀座線/末広町駅 歩5分\nＪＲ総武線/...,0,1,180000,15000,180000,0,2LDK,44.97
2,ウェルスクエア南大塚,東京都豊島区南大塚２,\nＪＲ山手線/大塚駅 歩7分\n東京メトロ丸ノ内線/新大塚駅 歩5分\nＪＲ山手線/巣鴨駅...,0,3,172000,12000,172000,172000,2LDK,40.59
3,ワールドレジデンス,東京都豊島区長崎５,\n西武池袋線/東長崎駅 歩5分\n東京メトロ副都心線/千川駅 歩14分\n都営大江戸線/落...,0,1,136000,3000,136000,136000,2LDK,38.97
6,ワールドレジデンス,東京都豊島区長崎５,\n西武池袋線/東長崎駅 歩5分\n東京メトロ副都心線/千川駅 歩14分\n都営大江戸線/落...,0,2,140000,3000,140000,140000,2LDK,38.97
...,...,...,...,...,...,...,...,...,...,...,...
167,東急大井町線 尾山台駅 2階建 築3年,東京都世田谷区等々力２,\n東急大井町線/尾山台駅 歩5分\n東急大井町線/等々力駅 歩7分\n東急大井町線/九品仏...,3,1,185000,3000,0,185000,2LDK,56.26
168,京王線 八幡山駅 3階建 新築,東京都杉並区上高井戸１,\n京王線/八幡山駅 歩2分\n京王線/芦花公園駅 歩10分\n京王線/上北沢駅 歩13分\n,0,1,187000,5000,187000,187000,2LDK,53.92
169,ロジェ代田,東京都世田谷区代田４,\n京王井の頭線/東松原駅 歩5分\n京王井の頭線/新代田駅 歩8分\n小田急線/世田谷代田...,3,1,155000,5500,0,155000,2LDK,40.86
170,京王井の頭線 東松原駅 2階建 築3年,東京都世田谷区代田４,\n京王井の頭線/東松原駅 歩5分\n京王井の頭線/新代田駅 歩8分\n京王線/明大前駅 歩...,3,1,155000,5500,0,155000,2LDK,40.86


In [10]:
#to_csv()を使って、データフレームをCSV出力する
df.to_csv("suumo_scraping4.csv",index=None,encoding="utf-8-sig")